In [1]:
import pandas as pd
import glob
from PIL import Image
import piexif
import  datetime
import os
from scipy.interpolate import interp1d
import calendar
from time import mktime
import time
import numpy as np

#### GPS Log

In [2]:
# Load the GPS log from Android's GPS Logger

df = pd.read_csv('20190627-170425.txt')
df['time'] = pd.to_datetime(df['time'])
df.head()

,type,time,latitude,longitude,accuracy (m),altitude (m),geoid_height (m),speed (m/s),bearing (deg),sat_used,sat_inview,name,desc
0,T,2019-06-27 21:04:25,37.268684,-80.373624,3,600.0,NaN,0.0,NaN,11,21,20190627-170425,GPS Logger: 20190627-170425
1,T,2019-06-27 21:04:26,37.268680,-80.373625,3,599.0,NaN,0.0,NaN,11,21,NaN,NaN
2,T,2019-06-27 21:04:27,37.268681,-80.373624,3,599.0,NaN,0.0,NaN,11,21,NaN,NaN
3,T,2019-06-27 21:04:28,37.268689,-80.373618,3,600.0,NaN,0.0,NaN,11,21,NaN,NaN
4,T,2019-06-27 21:04:29,37.268690,-80.373615,3,600.0,NaN,0.0,NaN,11,21,NaN,NaN


In [3]:
# Correct the log's UTC time (not GPS time!) to the correct timezone:

timezone = -4
df['time'] = df.time + datetime.timedelta(hours=timezone)

In [4]:
# Calculate a "seconds since start time" field, for interpolation

df['seconds_since_start'] = (df.time - df.time[0]).dt.total_seconds()

#### Images

In [5]:
# Define a helpful function to transform from decimal degrees to the tuples that EXIF needs

def dd_to_exif_tuple(dd):
    dd = np.abs(dd)
    d = int(np.floor(dd))
    m = int(np.floor(60 * (dd - d)))
    s = (dd - d - m/60) * 3600
    ss = int(np.floor(10000 * s))

    tup = ((d,1),(m,1),(ss,10000))
    return tup

In [7]:
indir = 'images/'
outdir = 'geotagged/'

# File time - Real time (time.gov time)
seconds_offset = 136.0

# Get image list
fns = glob.glob(indir + '*.jpg')
                
# Create empty dataframe
out_df = pd.DataFrame(columns=['filename','latitude','longitude','altitude'])

for fn in fns:
    # Open the image, and read its exif information
    with Image.open(fn) as im:
        exif_dict = piexif.load(im.info["exif"])
        
        # Pull the timestamp from EXIF
        image_exif_timestamp = exif_dict['0th'][306]
        image_exif_timestamp = datetime.datetime.strptime(image_exif_timestamp.decode('ascii'),'%Y:%m:%d %H:%M:%S')
        
        # Calculate diffrence between image time and GPS log starttime
        seconds_since_gps_start = (image_exif_timestamp - df.time[0]).total_seconds() - seconds_offset
        
        # Iterpolate latitude, longitude, and heights
        image_lat = np.interp(seconds_since_gps_start,df['seconds_since_start'],df['latitude'])
        image_lon = np.interp(seconds_since_gps_start,df['seconds_since_start'],df['longitude'])
        image_alt = np.interp(seconds_since_gps_start,df['seconds_since_start'],df['altitude (m)'])
        
        # Append this info into the dataframe
        out_df = out_df.append({'filename':fn,'latitude':image_lat,'longitude':image_lon,'altitude':image_alt},ignore_index=True)
        
        # Create the GPS info tag:
        gps_dict = {}
        gps_dict[0] = (2, 2, 0, 0)
        gps_dict[1] = b'N'
        gps_dict[2] = dd_to_exif_tuple(image_lat)
        gps_dict[3] = b'W'
        gps_dict[4] = dd_to_exif_tuple(image_lon)
        gps_dict[5] = 0
        gps_dict[6] = (int(1000 * image_alt),1000)
        exif_dict['GPS'] = gps_dict
        exif_bytes = piexif.dump(exif_dict)
        
        # Save a copy to the out directory
        im.save(outdir + fn.split('\\')[-1], "jpeg", exif=exif_bytes)
        
out_df.to_csv('geotagged_images.csv',index=False)